In [1]:
# ref: https://www.kaggle.com/jacklinggu/lstm-with-glove-embedding-public-lb-score-0-049

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from keras.models import Model, load_model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout,GlobalAveragePooling1D,Conv1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 40000
maxlen = 150



train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").values
print(y.shape)

Using TensorFlow backend.


(95851, 6)


In [2]:
list_sentences_train[:5]

array([ "Nonsense?  kiss off, geek. what I said is true.  I'll have your account terminated.",
       '"\n\n Please do not vandalize pages, as you did with this edit to W. S. Merwin. If you continue to do so, you will be blocked from editing.    "',
       '"\n\n ""Points of interest"" \n\nI removed the ""points of interest"" section you added because it seemed kind of spammy. I know you probably didn\'t mean to disobey the rules, but generally, a point of interest tends to be rather touristy, and quite irrelevant to an area culture. That\'s just my opinion, though.\n\nIf you want to reply, just put your reply here and add {{talkback|Jamiegraham08}} on my talkpage.   "',
       "Asking some his nationality is a Racial offence. Wow wasn't aware of it.  Blocking me has shown your support towards your community. Thanku for that",
       'The reader here is not going by my say so for ethereal vocal style and dark lyrical content. The cited sources in the External Links are saying those thi

In [3]:
print('test len',len(test))

test len 226998


In [4]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

(95851, 150) (226998, 150)


In [5]:
# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

maryse 54857
page195 133177
chrysler 28153
eläkeläiset 23457
accustaios 71516
u–th 145898


In [6]:
# read word2vec
# https://github.com/facebookresearch/MUSE
word_vec_dict = {}
with open('../wiki.multi.en.vec') as f:
    first_line_flag = True
    for line in f:
        if first_line_flag:
            first_line_flag= False
            continue
        v_list = line.split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v
print(len(word_vec_dict))


200000


In [7]:
print('Preparing embedding matrix')
EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing embedding matrix
Null word embeddings: 6145


In [8]:
from sklearn.metrics import log_loss,accuracy_score

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def get_lstm_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix],trainable=False)(inp)
    x = Bidirectional(LSTM(32, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
print('def model done')

def model done


In [9]:
from sklearn.model_selection import KFold
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((95851,6)),np.zeros((226998,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = get_lstm_model()
        batch_size = 64
        epochs = 15
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    print('all eval',eval_val(y,train_pred))
    return train_pred, test_pred


train_pred,test_pred = kf_train()
print(train_pred.shape,test_pred.shape)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/lstm_muse_2_csv_adj.gz", index=False, compression='gzip')
import pickle
with open('../features/lstm_muse_adj_2_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
print('done')


Train on 63900 samples, validate on 31951 samples
Epoch 1/15
63900/63900 [==============================] - 341s 5ms/step - loss: 0.0891 - acc: 0.9724 - val_loss: 0.0544 - val_acc: 0.9806
Epoch 2/15
63900/63900 [==============================] - 339s 5ms/step - loss: 0.0566 - acc: 0.9800 - val_loss: 0.0503 - val_acc: 0.9817
Epoch 3/15
63900/63900 [==============================] - 339s 5ms/step - loss: 0.0531 - acc: 0.9807 - val_loss: 0.0489 - val_acc: 0.9822
Epoch 4/15
63900/63900 [==============================] - 335s 5ms/step - loss: 0.0511 - acc: 0.9814 - val_loss: 0.0483 - val_acc: 0.9823
Epoch 5/15
63900/63900 [==============================] - 331s 5ms/step - loss: 0.0492 - acc: 0.9819 - val_loss: 0.0471 - val_acc: 0.9826
Epoch 6/15
63900/63900 [==============================] - 329s 5ms/step - loss: 0.0477 - acc: 0.9822 - val_loss: 0.0454 - val_acc: 0.9828
Epoch 7/15
63900/63900 [==============================] - 328s 5ms/step - loss: 0.0465 - acc: 0.9825 - val_loss: 0.0447 - 

Epoch 13/15
63901/63901 [==============================] - 328s 5ms/step - loss: 0.0419 - acc: 0.9837 - val_loss: 0.0446 - val_acc: 0.9832
Epoch 14/15
63901/63901 [==============================] - 327s 5ms/step - loss: 0.0409 - acc: 0.9841 - val_loss: 0.0443 - val_acc: 0.9835
Epoch 15/15
63901/63901 [==============================] - 328s 5ms/step - loss: 0.0404 - acc: 0.9843 - val_loss: 0.0443 - val_acc: 0.9833
Train on 63901 samples, validate on 31950 samples
Epoch 1/15
63901/63901 [==============================] - 330s 5ms/step - loss: 0.0870 - acc: 0.9737 - val_loss: 0.0552 - val_acc: 0.9801
Epoch 2/15
63901/63901 [==============================] - 329s 5ms/step - loss: 0.0549 - acc: 0.9803 - val_loss: 0.0522 - val_acc: 0.9809
Epoch 3/15
63901/63901 [==============================] - 329s 5ms/step - loss: 0.0520 - acc: 0.9812 - val_loss: 0.0506 - val_acc: 0.9813
Epoch 4/15
63901/63901 [==============================] - 329s 5ms/step - loss: 0.0498 - acc: 0.9818 - val_loss: 0.0492

In [10]:
train_pred[:10].round(3)

array([[ 0.316,  0.   ,  0.02 ,  0.   ,  0.031,  0.   ],
       [ 0.001,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.006,  0.   ,  0.   ,  0.   ,  0.001,  0.001],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.345,  0.004,  0.098,  0.002,  0.128,  0.01 ],
       [ 0.006,  0.   ,  0.001,  0.   ,  0.001,  0.   ],
       [ 0.171,  0.001,  0.01 ,  0.018,  0.019,  0.003],
       [ 0.001,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.014,  0.   ,  0.002,  0.   ,  0.003,  0.   ]])

In [11]:
y[:10]

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])